In [1]:
import json
import pandas as pd
import plotly.express as px
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import unicodedata

In [2]:

# Función para normalizar nombres de canciones (quita espacios extra y pasa a minúsculas)
def normalize(song):
    song = song.strip().lower()
    song = ''.join(c for c in unicodedata.normalize('NFKD', song) if not unicodedata.combining(c) or c == 'ñ')
    return song

# Cargar los archivos JSON
with open(r'C:\Users\Yo\Desktop\setlist\SetlistFM-Setlist-Evolution\album_data\Los Piojos.json', 'r', encoding='utf-8') as f:
    albums_data = json.load(f)

with open('Setlists_Piojos.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)

# Crear diccionarios para mapear cada canción a su álbum y a su color
song_to_album = {}
song_to_color = {}
for album in albums_data.get("albums", []):
    album_title = album.get("title", "Ineditos")
    color = album.get("color", "gray")
    for track in album.get("tracks", []):
        norm_track = normalize(track)
        # Si la canción aparece en varios álbumes, se asigna el primero encontrado
        if norm_track not in song_to_album:
            song_to_album[norm_track] = album_title
            song_to_color[norm_track] = color

# Crear un diccionario que mapea cada álbum a su color en formato HEX
album_to_color = {}
for album in albums_data.get("albums", []):
    album_to_color[album.get("title", "Ineditos")] = album.get("color", "#39df00")


# Ordenar los setlists por fecha (usando year, month, day)
setlists_data.sort(key=lambda s: date(s["year"], s["month"], s["day"]))

# Procesar cada setlist y contar acumulativamente
records = []
cumulative_counts = {}

for setlist in setlists_data:
    # Crear fecha en formato ISO
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    
    # Procesar la lista de canciones, separando por "/" si es necesario
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    
    # Actualizar los conteos acumulados
    for song in canciones:
        norm_song = normalize(song)
        cumulative_counts[norm_song] = cumulative_counts.get(norm_song, 0) + 1
    # Registrar el estado acumulado hasta la fecha actual
    for song, count in cumulative_counts.items():
        album = song_to_album.get(song, "Ineditos")
        color = song_to_color.get(song, "#000000")
        records.append({
            "fecha": current_date,
            "canción": song.title(),  # Capitalizamos para visualización
            "álbum": album,
            "color": color,
            "veces_tocada": count,

        })

# Crear DataFrame y agrupar para quedarnos con el último conteo de cada fecha
df = pd.DataFrame(records)


# Diccionario de colores corregido
colores_album = {
    "Ay Ay Ay": "#c41616",
    "3er Arco": "#ead936",
    "Chac Tu Chac": "#1e2830",
    "Azul": "#0047a5",
    "Verde Paisaje del Infierno": "#648e5c",
    "Maquina de Sangre": "#d57f5c",
    "Civilizacion": "#d7c391",
    "Ineditos": "#b3b3b3"
}

# Reemplazar los valores incorrectos en la columna 'color'
df["color"] = df["álbum"].map(colores_album)
df["álbum"] = df["álbum"].astype("category")

# Crear un DataFrame con todas las canciones en la fecha inicial con 0 reproducciones
fecha_inicio = df["fecha"].min()
df_inicio = df.copy()
df_inicio["fecha"] = fecha_inicio  # Asigna la fecha inicial
df_inicio["veces_tocada"] = 0      # Todas las canciones con 0

# Concatenamos con el DataFrame original
df_final = pd.concat([df_inicio, df], ignore_index=True)


In [ ]:


fechas_piojos = df_final[df_final["fecha"] is in 